# MaterialPointSolver.jl Showcase 03

Author: Zenan Huo <br>
Date: 25-Aug-2024 <br>

---

We use a 2D discs collision case to make a dashboard with MaterialPointSolver.jl.

- Presents how to modifiy the algorithm in kernel functions
- Illustrates how to invoke kernel functions written by yourself

In [74]:
using MaterialPointSolver
using MaterialPointGenerator
using CairoMakie
using CUDA
using ProgressMeter
using KernelAbstractions
using HDF5

!!! Add packages if they are not in your Julia `env`.

In [58]:
MaterialPointSolver.warmup(Val(:CUDA)) # optional

┌ Info: warming up on :CUDA [0] 🔥
└ @ MaterialPointSolverCUDAExt /home/zhuo/Workbench/MaterialPointSolver.jl/ext/CUDAExt/warmup_cuda.jl:23


In [59]:
figregular = MaterialPointSolver.tnr
figbold    = MaterialPointSolver.tnrb

init_NIC          = 9
init_phase        = 1
init_basis        = :uGIMP
init_grid_space_x = 0.005 # 0.05
init_grid_space_y = 0.005 # 0.05
init_grid_range_x = [-1, 1]
init_grid_range_y = [-1, 1]
init_mp_in_space  = 2
init_ρs           = 1000
init_ν            = 0.3
init_Es           = 1e3
init_Ks           = init_Es / (3 * (1 - 2 * init_ν))
init_Gs           = init_Es / (2 * (1 +     init_ν))
init_T            = 3
init_Te           = 0
init_ΔT           = 0.5 * init_grid_space_x / sqrt(init_Es / init_ρs) # 0.001
init_step         = (t = floor(init_T / init_ΔT / 700); t<10 ? 1 : t)
init_ϵ            = "FP64";

### Parameters Setup

In [60]:
args = UserArgs2D(
    Ttol         = init_T,
    Te           = 0,
    ΔT           = init_ΔT,
    time_step    = :fixed,
    FLIP         = 1,
    PIC          = 0,
    constitutive = :linearelastic,
    basis        = init_basis,
    animation    = false, 
    hdf5         = true,
    hdf5_step    = init_step,
    MVL          = false,
    device       = :CUDA,
    coupling     = :OS,
    scheme       = :MUSL,
    progressbar  = true,
    gravity      = 0,
    ζs           = 0,
    project_name = "2d_discs",
    project_path = @__DIR__,
    ϵ            = init_ϵ
)

DeviceArgs2D:
┬────────────
├─ project name    : 2d_discs
├─ project path    : /home/zhuo/Workbench/MaterialPointSolver.jl/examples
├─ precision       : FP64
├─ constitutive    : linearelastic
├─ basis method    : uGIMP
├─ mitigate vollock: false
└─ coupling scheme : OS


### Background Grid Setup

In [61]:
grid = UserGrid2D(
    ϵ     = init_ϵ,
    phase = init_phase,
    x1    = init_grid_range_x[1],
    x2    = init_grid_range_x[2],
    y1    = init_grid_range_y[1],
    y2    = init_grid_range_y[2],
    dx    = init_grid_space_x,
    dy    = init_grid_space_y,
    NIC   = init_NIC
)

DeviceGrid2D:
┬────────────
├─ phase  : 1
├─ NIC    : 9
├─ ϵ      : FP64
├─ x1 - x2: -1.0 - 1.0
├─ y1 - y2: -1.0 - 1.0
├─ dx - dy: 0.005 - 0.005
├─ nc     : 1.60e+05
└─ ni     : 1.61e+05


### Material Points Setup

In [62]:
dx = grid.dx / init_mp_in_space
dy = grid.dy / init_mp_in_space
ξ0 = meshbuilder(-0.5 + dx / 2 : dx : 0.5 - dx / 2,
                 -0.5 + dy / 2 : dy : 0.5 - dy / 2)
mp_num  = size(ξ0, 1)
a = findall(i->((-0.5≤ξ0[i, 1]≤-0.1)&&((ξ0[i, 2]+0.3)^2≤(0.04-(ξ0[i, 1]+0.3)^2)) ||
                ( 0.1≤ξ0[i, 1]≤ 0.5)&&((ξ0[i, 2]-0.3)^2≤(0.04-(ξ0[i, 1]-0.3)^2))),
    1:mp_num)
ξ0 = ξ0[a, :]
mp = UserParticle2D(
    ϵ     = init_ϵ,
    phase = 1,
    NIC   = init_NIC,
    dx    = dx,
    dy    = dy,
    ξ     = ξ0,
    ρs    = ones(size(ξ0, 1)) .* init_ρs
)
lb_id = findall(i -> mp.ξ[i, 2] < 0, 1:mp.np)
rt_id = findall(i -> mp.ξ[i, 2] > 0, 1:mp.np)
mp.vs[lb_id, :] .=  0.1
mp.vs[rt_id, :] .= -0.1
mp

DeviceParticle2D:
┬────────────────
├─ phase  : 1
├─ NIC    : 9
├─ ϵ      : FP64
├─ dx - dy: 0.0025 - 0.0025
└─ np     : 4.02e+04


### Particle Property Setup

In [63]:
nid = ones(mp.np)
attr = UserProperty(
    ϵ   = init_ϵ,
    nid = nid,
    ν   = [init_ν],
    Es  = [init_Es],
    Gs  = [init_Gs],
    Ks  = [init_Ks]
)

DeviceProperty:
┬──────────────
└─ ϵ: FP64


### Boundary Condition Nodes Index

In [64]:
vx_idx  = zeros(grid.ni)
vy_idx  = zeros(grid.ni)
tmp_idx = findall(i -> grid.ξ[i, 1] ≤ -1 || grid.ξ[i, 1] ≥ 1, 1:grid.ni)
tmp_idy = findall(i -> grid.ξ[i, 2] ≤ -1 || grid.ξ[i, 2] ≥ 1, 1:grid.ni)
vx_idx[tmp_idx] .= 1
vy_idx[tmp_idy] .= 1
bc = UserVBoundary2D(
    ϵ        = init_ϵ,
    vx_s_idx = vx_idx,
    vx_s_val = zeros(grid.ni),
    vy_s_idx = vy_idx,
    vy_s_val = zeros(grid.ni)
)

DeviceVBoundary2D:
┬─────────────────
└─ ϵ: FP64


### Rewrite kernel functions

- G2P process
- constitutive model

In [65]:
@kernel inbounds = true function testG2P!(
    grid::    DeviceGrid2D{T1, T2},
    mp  ::DeviceParticle2D{T1, T2}
) where {T1, T2}
    ix = @index(Global)
    if ix ≤ mp.np
        dF1 = dF2 = dF3 = dF4 = T2(0.0)
        @KAunroll for iy in Int32(1):Int32(mp.NIC)
            p2n = mp.p2n[ix, iy]
            ∂Nx = mp.∂Nx[ix, iy]
            ∂Ny = mp.∂Ny[ix, iy]
            # compute solid incremental deformation gradient
            dF1 += grid.Δus[p2n, 1] * ∂Nx
            dF2 += grid.Δus[p2n, 1] * ∂Ny
            dF3 += grid.Δus[p2n, 2] * ∂Nx
            dF4 += grid.Δus[p2n, 2] * ∂Ny
        end
        mp.ΔFs[ix, 1] = dF1
        mp.ΔFs[ix, 2] = dF2
        mp.ΔFs[ix, 3] = dF3
        mp.ΔFs[ix, 4] = dF4
        # compute strain increment 
        mp.Δϵijs[ix, 1] = dF1
        mp.Δϵijs[ix, 2] = dF4
        mp.Δϵijs[ix, 4] = dF2 + dF3
        # update strain tensor
        mp.ϵijs[ix, 1] += dF1
        mp.ϵijs[ix, 2] += dF4
        mp.ϵijs[ix, 4] += dF2 + dF3
        # deformation gradient matrix
        F1 = mp.F[ix, 1]; F2 = mp.F[ix, 2]; F3 = mp.F[ix, 3]; F4 = mp.F[ix, 4]      
        mp.F[ix, 1] = (dF1 + T2(1.0)) * F1 + dF2 * F3
        mp.F[ix, 2] = (dF1 + T2(1.0)) * F2 + dF2 * F4
        mp.F[ix, 3] = (dF4 + T2(1.0)) * F3 + dF3 * F1
        mp.F[ix, 4] = (dF4 + T2(1.0)) * F4 + dF3 * F2
    end
end

In [66]:
@kernel inbounds = true function testliE!(
    mp  ::DeviceParticle2D{T1, T2},
    attr::  DeviceProperty{T1, T2}
) where {T1, T2}
    ix = @index(Global)
    if ix ≤ mp.np
        pid = attr.nid[ix]
        Ks  = attr.Ks[pid]
        Gs  = attr.Gs[pid]
        # linear elastic
        Dt = Ks + T2(4/3) * Gs
        Dd = Ks - T2(2/3) * Gs
        mp.σij[ix, 1] += Dt * mp.Δϵijs[ix, 1] + Dd * mp.Δϵijs[ix, 2] + Dd * mp.Δϵijs[ix, 3]
        mp.σij[ix, 2] += Dd * mp.Δϵijs[ix, 1] + Dt * mp.Δϵijs[ix, 2] + Dd * mp.Δϵijs[ix, 3]
        mp.σij[ix, 3] += Dd * mp.Δϵijs[ix, 1] + Dd * mp.Δϵijs[ix, 2] + Dt * mp.Δϵijs[ix, 3]
        mp.σij[ix, 4] += Gs * mp.Δϵijs[ix, 4]
        # update mean stress tensor
        σm = (mp.σij[ix, 1] + mp.σij[ix, 2] + mp.σij[ix, 3]) * T2(1/3)
        mp.σm[ix] = σm
        # update deviatoric stress tensor
        mp.sij[ix, 1] = mp.σij[ix, 1] - σm
        mp.sij[ix, 2] = mp.σij[ix, 2] - σm
        mp.sij[ix, 3] = mp.σij[ix, 3] - σm
        mp.sij[ix, 4] = mp.σij[ix, 4]
    end
end

- Put them together in an MPM procedure

In [67]:
function testprocedure!(
    args::     DeviceArgs{T1, T2}, 
    grid::     DeviceGrid{T1, T2}, 
    mp  :: DeviceParticle{T1, T2}, 
    attr:: DeviceProperty{T1, T2},
    bc  ::DeviceVBoundary{T1, T2},
    ΔT  ::T2,
    Ti  ::T2,
        ::Val{:OS},
        ::Val{:MUSL}
) where {T1, T2}
    G = Ti < args.Te ? args.gravity / args.Te * Ti : args.gravity
    dev = getBackend(Val(args.device))
    resetgridstatus_OS!(dev)(ndrange=grid.ni, grid)
    args.device == :CPU && args.basis == :uGIMP ? 
        resetmpstatus_OS_CPU!(dev)(ndrange=mp.np, grid, mp, Val(args.basis)) :
        resetmpstatus_OS!(dev)(ndrange=mp.np, grid, mp, Val(args.basis))
    P2G_OS!(dev)(ndrange=mp.np, grid, mp, G)
    solvegrid_OS!(dev)(ndrange=grid.ni, grid, bc, ΔT, args.ζs)
    doublemapping1_OS!(dev)(ndrange=mp.np, grid, mp, attr, ΔT, args.FLIP, args.PIC)
    doublemapping2_OS!(dev)(ndrange=mp.np, grid, mp)
    doublemapping3_OS!(dev)(ndrange=grid.ni, grid, bc, ΔT)
    testG2P!(dev)(ndrange=mp.np, grid, mp)
    testliE!(dev)(ndrange=mp.np, mp, attr)                
    return nothing
end

testprocedure! (generic function with 1 method)

### MPM Solver

!!! Here we identify the workflow as `testprocedure!`.

In [68]:
materialpointsolver!(args, grid, mp, attr, bc, workflow=testprocedure!)

[▲ I/O: uploading [≈ 0.0 GiB] → :CUDA [0]


┌ Info: 2d_discs [2D/CUDA]
│ ────────────────┬─────────────┬─────────────────
│ ΔT  : 2.50e-03s │ PIC :  0.00 │ scheme   : MUSL
│ Ttol: 3.00e+00s │ FLIP:  1.00 │ coupling : OS
│ pts : 4.02e+04  │ ζs  :  0.00 │ animation: false
│ nds : 1.61e+05  │ ζw  :  0.00 │ precision: FP64
│ MVL :    false  │ HDF5:  true │ material : L-E
│ ────────────────┴─────────────┴─────────────────
└ @ MaterialPointSolver /home/zhuo/Workbench/MaterialPointSolver.jl/src/toolkits/terminaltxt.jl:49
[ Info: solving 100% ■■■■■■■■■■■■  Time: 0:00:04


[▼ I/O: downloading from :CUDA [0] → host
[• I/O: free device [0] memory


┌ Info: performance
│ ────────────────────
│ wtime: 00:00:05
│ iters: 1.20e+03
│ speed: 2.54e+02 it/s
│ ────────────────────
└ @ MaterialPointSolver /home/zhuo/Workbench/MaterialPointSolver.jl/src/toolkits/terminaltxt.jl:78


### Post processing

In [71]:
# dashboard (it will take some time ...)
@views let
    hdf5_path  = joinpath(args.project_path, args.project_name, "$(args.project_name).h5")
    fid        = h5open(hdf5_path, "r")
    video_step = 5
    video_fps  = 30
    itr        = (read(fid["FILE_NUM"])-1) |> Int64
    mp_pos     = Observable(fid["mp_coords0"] |> read)
    mp_num     = length(mp_pos[][:, 1])
    mp_Vsc     = Observable(zeros(mp_num))
    mp_σm      = Observable(zeros(mp_num))
    e1         = Observable(Point2f[(0.0, 0.0)])
    e2         = Observable(Point2f[(0.0, 0.0)])
    e3         = Observable(Point2f[(0.0, 0.0)])
    anno       = Observable(0.0)

    fig = Figure(size=(1080, 820), fontsize=25, fonts=(; regular=figregular, 
        bold=figbold))
    Label(fig[0, 1:4], "Two elastic bodies collision", fontsize=30)
    ax1 = Axis(fig[1, 1], aspect=DataAspect(), xticks=(-0.5:0.2:0.5), 
        yticks=(-0.5:0.2:0.5))
    ax2 = Axis(fig[1, 3], aspect=DataAspect(), xticks=(-0.5:0.2:0.5), 
        yticks=(-0.5:0.2:0.5))
    ax3 = Axis(fig[2, 1:4], aspect=AxisAspect(4), xticks=(0:0.5:3.0), 
        yticks=(0.5:1:2.5))
    
    p1 = scatter!(ax1, mp_pos, color=mp_σm, colormap=:darktest, markersize=2, 
        colorrange=(-300, 100))
    Colorbar(fig[1, 2], p1, width=20, spinewidth=0, label="Mean stress")
    p2 = scatter!(ax2, mp_pos, color=mp_Vsc, colormap=:darktest, markersize=2 ,
        colorrange=(0, 0.3))
    Colorbar(fig[1, 4], p2, width=20, spinewidth=0, label="Σ Veloclty")

    p3 = scatterlines!(ax3, e1, markersize=0, linewidth=3, color=:blue )
    p4 = scatterlines!(ax3, e2, markersize=0, linewidth=3, color=:green)
    p5 = scatterlines!(ax3, e3, markersize=0, linewidth=3, color=:red  )
    axislegend(ax3, [p3, p4, p5], ["kinetic", "strain", "total"], "Energy", 
        labelsize=20)
    vlines!(ax3, anno  , color=:black, linewidth=1)
    vlines!(ax3, 1.5858, color=:red  , linewidth=2, linestyle=:dash)
    text!(ax3, 0.8, 1.3, text="analytical contact", fontsize=20)

    lines!(ax1, [0, 0], [-0.2, 0.2], color=:red, linewidth=3)
    lines!(ax1, [-0.2, 0.2], [0, 0], color=:red, linewidth=3)
    lines!(ax2, [0, 0], [-0.2, 0.2], color=:red, linewidth=3)
    lines!(ax2, [-0.2, 0.2], [0, 0], color=:red, linewidth=3)
    xlims!(ax1, -0.60, 0.6)
    ylims!(ax1, -0.60, 0.6)
    xlims!(ax2, -0.60, 0.6)
    ylims!(ax2, -0.60, 0.6)
    xlims!(ax3, -0.1, 3.5)
    ylims!(ax3, -0.1, 3.0)
    p = Progress(length(1:video_step:itr)-1; 
        desc="\e[1;36m[ Info:\e[0m $(lpad("video", 7))", color=:white, barlen=12, 
        barglyphs=BarGlyphs(" ◼◼  "))
    CairoMakie.record(fig, joinpath(args.project_path, args.project_name, "$(args.project_name).mp4"),
        1:video_step:itr; framerate=video_fps) do i
        mp_σij = fid["group$(i)/stress"]   |> read
        mp_Ms  = fid["group$(i)/mass_s"]  |> read
        mp_ϵij = fid["group$(i)/strain_s"] |> read
        mp_Vol = fid["group$(i)/volume"]   |> read
        mp_Vs  = fid["group$(i)/velocity_s"]   |> read
        time   = fid["group$(i)/time"]  |> read
        Vsc  = sqrt.(mp_Vs[:, 1].^2 .+mp_Vs[:, 2].^2)
        σm   = (mp_σij[:, 1].+mp_σij[:, 2].+mp_σij[:, 3])./3
        tmp1 = Point2f(time, sum(sum(0.5.*mp_Vs.^2 .*mp_Ms, dims=2)))
        tmp2 = Point2f(time, sum(0.5.*mp_σij.*mp_ϵij.*mp_Vol))
        tmp3 = Point2f(time, tmp1[2]+tmp2[2])
        mp_pos[] = fid["group$(i)/coords"] |> read
        mp_Vsc[] = Vsc
        mp_σm[]  = σm
        anno[]   = time
        push!(e1[], tmp1)
        push!(e2[], tmp2)
        push!(e3[], tmp3)
        next!(p)
    end
    close(fid)
end

[ Info:   video 100% ◼◼◼◼◼◼◼◼◼◼◼◼  Time: 0:03:09


<video width="640" height="360" controls>
  <source src="https://raw.githubusercontent.com/LandslideSIM/.github/main/assets/MaterialPointSolver.jl/2d_discs.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>